# Step: Read Documents - batch_read

**Pipeline**: batch_read  
**Step**: read_documents  
**Job Template**: simple_read_volumes

**Purpose**: Simple batch read from Volumes and store in Bronze Delta table

**Inputs**:
- Source: Configuration-driven from `batch_read_config.yaml`
- Format: PDF, JSON, CSV, Parquet, etc.

**Outputs**:
- Bronze table: `${catalog}.${bronze_schema}.batch_read_documents_bronze`

In [ ]:
# Import required libraries
import json
from pyspark.sql import functions as F
from databricks import dbutils

In [ ]:
# Load configuration from task variables (set by setup task)
config_json = dbutils.jobs.taskValues.get(taskKey="setup", key="config")
config = json.loads(config_json)

# Get globals
globals_config = config["globals"]
catalog = globals_config["catalog"]
bronze_schema = globals_config["bronze_schema"]

# Get step configuration
pipeline_config = config["pipelines"]["batch_read"]
step_config = pipeline_config["pdf"]["standard"]["steps"][0]  # First step: read_documents

source_path = step_config["source"]
output_table = step_config["output"]

print(f"Source: {source_path}")
print(f"Output: {output_table}")

In [ ]:
# Simple batch read from Volumes
df = (
    spark.read
    .format("binaryFile")  # Read as binary for document processing
    .load(source_path)
)

# Add metadata columns
df_enriched = (
    df
    .withColumn("file_name", F.element_at(F.split(F.col("path"), "/"), -1))
    .withColumn("file_path", F.col("path"))
    .withColumn("file_content", F.col("content"))
    .withColumn("_ingestion_timestamp", F.current_timestamp())
    .withColumn("_processing_date", F.current_date())
    .select("file_name", "file_path", "file_content", "modificationTime", "length", "_ingestion_timestamp", "_processing_date")
)

print(f"Records read: {df_enriched.count()}")
df_enriched.printSchema()

In [ ]:
# Write to Delta table
(
    df_enriched.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(output_table)
)

print(f"✅ Documents written to {output_table}")